In [1]:
import keras

Using TensorFlow backend.


## Load necessary modules

In [6]:
# show images inline
%matplotlib inline

# automatically reload modules when they have changed
%load_ext autoreload
%autoreload 2

# import keras
import matplotlib.pyplot as plt
import keras



# import keras_retinanet
from keras_retinanet import models
from keras_retinanet.utils.image import read_image_bgr, preprocess_image, resize_image
from keras_retinanet.utils.visualization import draw_box, draw_caption
from keras_retinanet.utils.colors import label_color

# import miscellaneous modules
import cv2
import os
import numpy as np
import time

# set tf backend to allow memory to grow, instead of claiming everything
import tensorflow as tf


def get_session():
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)

# use this environment flag to change which GPU to use
#os.environ["CUDA_VISIBLE_DEVICES"] = "1"

# set the modified tf session as backend in keras
keras.backend.tensorflow_backend.set_session(get_session())

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load RetinaNet model

In [5]:
!ls

CONTRIBUTORS.md
ISSUE_TEMPLATE.md
LICENSE
README.md
ResNet50RetinaNet.ipynb
annotations.txt
build
class_names.csv
examples
images
keras_retinanet
logs
setup.cfg
setup.py
snapshots
tests
weapon


In [7]:


model_path = os.path.join('..', 'snapshots', 'resnet50_csv_05.h5')
model = models.load_model(model_path, backbone_name='resnet50')
print(model.summary())

ModuleNotFoundError: No module named 'keras_resnet'

In [ ]:
# adjust this to point to your downloaded/trained model
# mode|zls can be downloaded here: https://github.com/fizyr/keras-retinanet/releases
#model_path = os.path.join('..', 'snapshots', 'resnet50_coco_best_v2.1.0.h5')

# load retinanet model
#model = models.load_model(model_path, backbone_name='resnet50')

# if the model is not converted to an inference model, use the line below
# see: https://github.com/fizyr/keras-retinanet#converting-a-training-model-to-inference-model
model = models.convert_model(model)

#print(model.summary())

# load label to names mapping for visualization purposes
labels_to_names = {0: 'bullet', 1: 'knife', 2: 'container', 3: 'gun', 4: 'screwdriver'}

## Run detection on example

In [ ]:
#labels_to_names = {0: 'bolt', 1: 'screw', 2: 'nail', 3: 'spring', 4: 'nut'}
#model = models.convert_model(model)

#model.predict_on_batch(np.expand_dims(image, axis=0))


In [ ]:
# load image
#image = read_image_bgr('000000008021.jpg')
#image = read_image_bgr('/Users/ScottsPC/Desktop/images/bear.jpg')
image = read_image_bgr('/Users/ScottsPC/Desktop/keras-retinanet/weapon/JPEGImages/bullet1002.jpg')
#image = read_image_bgr('/Users/ScottsPC/Desktop/keras-retinanet/images.jpg')
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

# process image
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
#boxes, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

#print(labels)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.3:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    caption = "{} {:.2f}".format(labels_to_names[label], score)
    draw_caption(draw, b, caption)
    
plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.show()

In [ ]:
#def predict(filename):
image = read_image_bgr('/Users/ScottsPC/Desktop/images/cafe.jpg')
# copy to draw on
draw = image.copy()
draw = cv2.cvtColor(draw, cv2.COLOR_BGR2RGB)

# preprocess image for network
image = preprocess_image(image)
image, scale = resize_image(image)

# process image
start = time.time()
boxes, scores, labels = model.predict_on_batch(np.expand_dims(image, axis=0))
print("processing time: ", time.time() - start)

# correct for image scale
boxes /= scale

# visualize detections
for box, score, label in zip(boxes[0], scores[0], labels[0]):
    # scores are sorted so we can break
    if score < 0.6:
        break
        
    color = label_color(label)
    
    b = box.astype(int)
    draw_box(draw, b, color=color)
    
    caption = "{} {:.2f}".format(labels_to_names[label], score)
    draw_caption(draw, b, caption)
    
plt.figure(figsize=(15, 15))
plt.axis('off')
plt.imshow(draw)
plt.show()

In [ ]:
scores